In [37]:
!pip install pyeda


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from pyeda.inter import *
from pyeda.boolalg.expr import Variable, Complement, OrOp, AndOp
from collections import defaultdict

def parse_condition(condition):
    """
    Парсит условие и возвращает переменную, оператор и значение.
    """
    condition = condition.strip()
    if '<=' in condition:
        variable, value = condition.split('<=', 1)
        operator = '<='
    elif '>=' in condition:
        variable, value = condition.split('>=', 1)
        operator = '>='
    elif '<' in condition:
        variable, value = condition.split('<', 1)
        operator = '<'
    elif '>' in condition:
        variable, value = condition.split('>', 1)
        operator = '>'
    elif '==' in condition:
        variable, value = condition.split('==', 1)
        operator = '=='
    else:
        raise ValueError(f"Unknown operator in condition: {condition}")
    return variable.strip(), operator.strip(), float(value.strip())

class RuleTransformer:
    def __init__(self, data):
        self.data = data
        self.condition_vars = {}
        self.variables = set()
        self.parse_rules()

    def parse_rules(self):
        """
        Парсит все правила и собирает уникальные условия.
        """
        self.parsed_data = []
        for conditions, result in self.data:
            parsed_conditions = []
            for cond in conditions:
                var, op, val = parse_condition(cond)
                var_name = f"{var}_{op}_{val}"
                self.variables.add(var_name)
                self.condition_vars[var_name] = (var, op, val)
                parsed_conditions.append(var_name)
            self.parsed_data.append((parsed_conditions, result))

    def build_expressions(self):
        """
        Строит булевы выражения для каждого правила.
        """
        self.expressions = []
        for conditions, result in self.parsed_data:
            expr_rule = expr(1)  # Начинаем с True
            for var_name in conditions:
                bv = exprvar(var_name)
                expr_rule = expr_rule & bv
            if result == 1:
                self.expressions.append(expr_rule)
            else:
                # Если результат 0, добавляем отрицание правила
                self.expressions.append(~expr_rule)

    def minimize(self):
        """
        Применяет минимизацию к булевым функциям.
        """
        if not self.expressions:
            self.minimized_expr = expr(0)  # Всегда False
        else:
            combined_expr = And(*self.expressions)
            combined_expr = combined_expr.to_dnf()  # Преобразуем в ДНФ
            minimized_exprs = espresso_exprs(combined_expr)
            self.minimized_expr = minimized_exprs[0]

    def expr_to_conditions(self, expr):
        """
        Рекурсивно преобразует минимизированное выражение обратно в условия.
        """
        if expr.is_one():
            return []  # Нет условий, всегда True
        elif expr.is_zero():
            return None  # Всегда False
        elif isinstance(expr, Variable):
            # expr — это переменная
            var_name = str(expr)
            condition = self.get_condition_from_var(var_name)
            return [condition]
        elif isinstance(expr, Complement):
            # expr — это отрицание переменной
            print(expr, type(expr), dir(Complement))
            var_name = expr
            condition = self.get_negated_condition_from_var(var_name)
            print(condition)
            return [condition]
        elif isinstance(expr, AndOp):
            # expr — это конъюнкция (AND)
            conditions = []
            for arg in expr.xs:
                sub_conditions = self.expr_to_conditions(arg)
                if sub_conditions is None:
                    return None
                conditions.extend(sub_conditions)
            return conditions
        elif isinstance(expr, OrOp):
            # expr — это дизъюнкция (OR)
            rules = []
            for arg in expr.xs:
                sub_conditions = self.expr_to_conditions(arg)
                if sub_conditions is not None:
                    rules.append(sub_conditions)
            return rules
        else:
            # Другие случаи
            return None

    def get_condition_from_var(self, var_name):
        """
        Получает строковое представление условия из имени булевой переменной.
        """
        var, op, val = self.condition_vars[var_name]
        return f"{var} {op} {val}"

    def get_negated_condition_from_var(self, var_name):
        """
        Получает строковое представление отрицания условия из имени булевой переменной.
        """
        var, op, val = self.condition_vars[var_name]
        # Определяем обратный оператор
        negated_op = {
            '<=': '>',
            '>=': '<',
            '<': '>=',
            '>': '<=',
            '==': '!='
        }.get(op)
        if not negated_op:
            raise ValueError(f"Unsupported operator for negation: {op}")
        return f"{var} {negated_op} {val}"

    def transform(self):
        self.build_expressions()
        self.minimize()
        # Преобразуем минимизированное выражение обратно в правила
        expr = self.minimized_expr
        # Для отладки можно распечатать минимизированное выражение
        # print(f"Минимизированное выражение: {expr}")
        rules = self.expr_to_conditions(expr)
        simplified_rules = []
        if rules is None or not rules:
            return []
        if all(isinstance(rule, list) for rule in rules):
            # Несколько правил
            for conditions in rules:
                simplified_rules.append((conditions, 1))
        else:
            # Одно правило
            simplified_rules.append((rules, 1))
        return simplified_rules


In [3]:
def test_case_1():
    input_data = [
        (["smoking <= 0.16", "diabets <= 0.95", "age <= 62.31", "age <= 57.55"], 0),
        (["smoking <= 0.16", "diabets <= 0.95", "age <= 62.31", "age > 57.55"], 1),
        (["smoking <= 0.16", "diabets <= 0.95", "age > 62.31", "sex_male <= 0.34"], 0),
        (["smoking <= 0.16", "diabets <= 0.95", "age > 62.31", "sex_male > 0.34"], 1),
        (["smoking <= 0.16", "diabets > 0.95", "sex_male <= 0.92", "ОХС <= 4.62"], 1),
        (["smoking <= 0.16", "diabets > 0.95", "sex_male <= 0.92", "ОХС > 4.62"], 1),
        (["smoking <= 0.16", "diabets > 0.95", "sex_male > 0.92", "age <= 72.49"], 1),
        (["smoking <= 0.16", "diabets > 0.95", "sex_male > 0.92", "age > 72.49"], 1),
        (["smoking > 0.16", "ОХС <= 3.34"], 0),
        (["smoking > 0.16", "ОХС > 3.34", "ОХС <= 6.16", "age <= 62.18"], 1),
        (["smoking > 0.16", "ОХС > 3.34", "ОХС <= 6.16", "age > 62.18"], 1),
        (["smoking > 0.16", "ОХС > 3.34", "ОХС > 6.16", "sex_male <= 0.84"], 1),
        (["smoking > 0.16", "ОХС > 3.34", "ОХС > 6.16", "sex_male > 0.84"], 1),
    ]

    expected_output = [
        (["smoking == 0", "diabets == 0", "age <= 62.31", "age <= 57.55"], 0),
        (["smoking == 0", "diabets == 0", "age <= 62.31", "age > 57.55"], 1),
        (["smoking == 0", "diabets == 0", "age > 62.31", "sex_male <= 0.34"], 0),
        (["smoking == 0", "diabets == 0", "age > 62.31", "sex_male > 0.34"], 1),
        (["smoking == 0", "diabets == 1"], 1),
        (["smoking == 1", "ОХС <= 3.34"], 0),
        (["smoking == 1", "ОХС > 3.34"], 1),
    ]

    processed = RuleTransformer(data=input_data).transform()

    # Выводим полученные правила для сравнения
    print("Полученные правила:")
    for rule in processed:
        print(rule)

    print("\nОжидаемый результат:")
    for rule in expected_output:
        print(rule)

# test_case_1()


In [ ]:
from pyeda.boolalg.expr import AndOp
print(dir(AndOp))

In [40]:
print(dir(expr))

['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__getstate__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
